# Actions

> Keyboard navigation focus zone and action factories for the card stack.

In [ ]:
#| default_exp keyboard.actions

In [ ]:
#| export
from typing import Optional, Tuple

from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone
from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
from cjm_fasthtml_keyboard_navigation.core.navigation import ScrollOnly

from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.js.core import global_callback_name

## create_card_stack_focus_zone

Creates a `FocusZone` configured for the card stack viewport. Uses `ScrollOnly`
navigation since all navigation is handled via HTMX button triggers, not the
keyboard library's built-in item navigation.

In [ ]:
#| export
def create_card_stack_focus_zone(
    ids: CardStackHtmlIds,  # HTML IDs for this card stack instance
    on_focus_change: Optional[str] = None,  # JS callback name on focus change
    hidden_input_prefix: Optional[str] = None,  # Prefix for keyboard nav hidden inputs
    data_attributes: Tuple[str, ...] = (),  # Data attributes to track on focused items
) -> FocusZone:  # Configured focus zone for the card stack
    """Create a focus zone for a card stack viewport."""
    return FocusZone(
        id=ids.card_stack,
        item_selector="[data-card-role='focused']",
        navigation=ScrollOnly(),
        data_attributes=data_attributes,
        zone_focus_classes=(),
        item_focus_classes=(),
        on_focus_change=on_focus_change or "",
        hidden_input_prefix=hidden_input_prefix or f"{ids.prefix}-focused",
    )

In [ ]:
# Test create_card_stack_focus_zone
ids = CardStackHtmlIds(prefix="cs0")
zone = create_card_stack_focus_zone(ids)

assert zone.id == "cs0-card-stack"
assert zone.item_selector == "[data-card-role='focused']"
assert isinstance(zone.navigation, ScrollOnly)
assert zone.zone_focus_classes == ()
assert zone.item_focus_classes == ()
assert zone.hidden_input_prefix == "cs0-focused"
print("Focus zone default tests passed!")

Focus zone default tests passed!


In [ ]:
# Test with custom parameters
zone = create_card_stack_focus_zone(
    ids,
    on_focus_change="onCardFocusChange",
    hidden_input_prefix="sd-decomp-focused",
    data_attributes=("segment-index",),
)
assert zone.on_focus_change == "onCardFocusChange"
assert zone.hidden_input_prefix == "sd-decomp-focused"
assert zone.data_attributes == ("segment-index",)
print("Focus zone custom parameter tests passed!")

Focus zone custom parameter tests passed!


In [ ]:
# Test multi-instance uniqueness
ids_a = CardStackHtmlIds(prefix="text")
ids_b = CardStackHtmlIds(prefix="vad")
zone_a = create_card_stack_focus_zone(ids_a)
zone_b = create_card_stack_focus_zone(ids_b)
assert zone_a.id != zone_b.id
assert zone_a.hidden_input_prefix != zone_b.hidden_input_prefix
print("Multi-instance focus zone tests passed!")

Multi-instance focus zone tests passed!


## create_card_stack_nav_actions

Creates the standard keyboard navigation actions for a card stack:

- **ArrowUp/Down**: Navigate prev/next item (HTMX button trigger)
- **Ctrl+ArrowUp/Down**: Page jump (JS callback)
- **Ctrl+Shift+ArrowUp/Down**: First/last item (JS callback)
- **`[`/`]`**: Narrow/widen viewport (JS callback)
- **`-`/`=`**: Decrease/increase scale (JS callback)

JS callbacks use prefix-unique global names that map to the namespaced
`window.cardStacks[prefix]` functions.

In [ ]:
#| export
def create_card_stack_nav_actions(
    zone_id: str,  # Focus zone ID to restrict actions to
    button_ids: CardStackButtonIds,  # Button IDs for HTMX triggers
    config: CardStackConfig,  # Config (for prefix-unique callback names)
    disable_in_modes: Tuple[str, ...] = (),  # Mode names that disable navigation
) -> Tuple[KeyAction, ...]:  # Standard card stack navigation actions
    """Create standard keyboard navigation actions for a card stack."""
    zone_ids = (zone_id,)
    not_modes = disable_in_modes if disable_in_modes else ()
    prefix = config.prefix

    return (
        # --- Item navigation (HTMX triggers) ---
        KeyAction(
            key="ArrowUp",
            htmx_trigger=button_ids.nav_up,
            zone_ids=zone_ids,
            not_modes=not_modes,
            description="Previous item",
            hint_group="Navigation",
        ),
        KeyAction(
            key="ArrowDown",
            htmx_trigger=button_ids.nav_down,
            zone_ids=zone_ids,
            not_modes=not_modes,
            description="Next item",
            hint_group="Navigation",
        ),

        # --- Page jump (JS callbacks, prefix-unique) ---
        KeyAction(
            key="ArrowUp",
            modifiers=frozenset({"ctrl"}),
            js_callback=global_callback_name(prefix, "jumpPageUp"),
            zone_ids=zone_ids,
            not_modes=not_modes,
            description="Page up",
            hint_group="Navigation",
        ),
        KeyAction(
            key="ArrowDown",
            modifiers=frozenset({"ctrl"}),
            js_callback=global_callback_name(prefix, "jumpPageDown"),
            zone_ids=zone_ids,
            not_modes=not_modes,
            description="Page down",
            hint_group="Navigation",
        ),

        # --- First/last item (JS callbacks, prefix-unique) ---
        KeyAction(
            key="ArrowUp",
            modifiers=frozenset({"ctrl", "shift"}),
            js_callback=global_callback_name(prefix, "jumpToFirstItem"),
            zone_ids=zone_ids,
            not_modes=not_modes,
            description="First item",
            hint_group="Navigation",
        ),
        KeyAction(
            key="ArrowDown",
            modifiers=frozenset({"ctrl", "shift"}),
            js_callback=global_callback_name(prefix, "jumpToLastItem"),
            zone_ids=zone_ids,
            not_modes=not_modes,
            description="Last item",
            hint_group="Navigation",
        ),

        # --- Width adjustment (available in any mode) ---
        KeyAction(
            key="[",
            js_callback=global_callback_name(prefix, "decreaseWidth"),
            zone_ids=zone_ids,
            description="Narrower",
            hint_group="View",
        ),
        KeyAction(
            key="]",
            js_callback=global_callback_name(prefix, "increaseWidth"),
            zone_ids=zone_ids,
            description="Wider",
            hint_group="View",
        ),

        # --- Scale adjustment (available in any mode) ---
        KeyAction(
            key="-",
            js_callback=global_callback_name(prefix, "decreaseScale"),
            zone_ids=zone_ids,
            description="Smaller",
            hint_group="View",
        ),
        KeyAction(
            key="=",
            js_callback=global_callback_name(prefix, "increaseScale"),
            zone_ids=zone_ids,
            description="Larger",
            hint_group="View",
        ),
    )

In [ ]:
from cjm_fasthtml_card_stack.core.config import CardStackConfig, _reset_prefix_counter

# Test create_card_stack_nav_actions returns 10 actions
_reset_prefix_counter()
config = CardStackConfig()
btn_ids = CardStackButtonIds(prefix=config.prefix)
ids = CardStackHtmlIds(prefix=config.prefix)
zone = create_card_stack_focus_zone(ids)

actions = create_card_stack_nav_actions(
    zone_id=zone.id,
    button_ids=btn_ids,
    config=config,
)

assert len(actions) == 10
print(f"Got {len(actions)} actions")

Got 10 actions


In [ ]:
# Test ArrowUp/Down actions use HTMX triggers
arrow_up = actions[0]
arrow_down = actions[1]

assert arrow_up.key == "ArrowUp"
assert arrow_up.htmx_trigger == btn_ids.nav_up
assert arrow_up.zone_ids == (zone.id,)

assert arrow_down.key == "ArrowDown"
assert arrow_down.htmx_trigger == btn_ids.nav_down
print("Arrow key action tests passed!")

Arrow key action tests passed!


In [ ]:
# Test Ctrl+Arrow actions use prefix-unique JS callbacks
page_up = actions[2]
page_down = actions[3]

assert page_up.js_callback == "cs0_jumpPageUp"
assert page_down.js_callback == "cs0_jumpPageDown"
assert page_up.modifiers == frozenset({"ctrl"})
print("Page jump callback tests passed!")

Page jump callback tests passed!


In [ ]:
# Test Ctrl+Shift+Arrow actions use prefix-unique JS callbacks
first_item = actions[4]
last_item = actions[5]

assert first_item.js_callback == "cs0_jumpToFirstItem"
assert last_item.js_callback == "cs0_jumpToLastItem"
assert first_item.modifiers == frozenset({"ctrl", "shift"})
print("First/last callback tests passed!")

First/last callback tests passed!


In [ ]:
# Test width adjustment actions use prefix-unique JS callbacks
narrow = actions[6]
widen = actions[7]

assert narrow.key == "["
assert narrow.js_callback == "cs0_decreaseWidth"
assert widen.key == "]"
assert widen.js_callback == "cs0_increaseWidth"
print("Width adjustment callback tests passed!")

Width adjustment callback tests passed!


In [ ]:
# Test scale adjustment actions use prefix-unique JS callbacks
scale_down = actions[8]
scale_up = actions[9]

assert scale_down.key == "-"
assert scale_down.js_callback == "cs0_decreaseScale"
assert scale_up.key == "="
assert scale_up.js_callback == "cs0_increaseScale"
print("Scale adjustment callback tests passed!")

# Test disable_in_modes filtering
actions_with_modes = create_card_stack_nav_actions(
    zone_id=zone.id,
    button_ids=btn_ids,
    config=config,
    disable_in_modes=("split", "edit"),
)

# Navigation actions should have not_modes set
assert actions_with_modes[0].not_modes == ("split", "edit")
assert actions_with_modes[1].not_modes == ("split", "edit")
assert actions_with_modes[2].not_modes == ("split", "edit")

# Width and scale actions should NOT have not_modes (available in any mode)
assert actions_with_modes[6].not_modes == None
assert actions_with_modes[7].not_modes == None
assert actions_with_modes[8].not_modes == None
assert actions_with_modes[9].not_modes == None
print("Mode filtering tests passed!")

Scale adjustment callback tests passed!
Mode filtering tests passed!


In [ ]:
# Test multi-instance: different prefixes produce different callback names
config_a = CardStackConfig(prefix="text")
config_b = CardStackConfig(prefix="vad")
btn_a = CardStackButtonIds(prefix="text")
btn_b = CardStackButtonIds(prefix="vad")
ids_a = CardStackHtmlIds(prefix="text")
ids_b = CardStackHtmlIds(prefix="vad")

actions_a = create_card_stack_nav_actions(ids_a.card_stack, btn_a, config_a)
actions_b = create_card_stack_nav_actions(ids_b.card_stack, btn_b, config_b)

# HTMX triggers are different (different button IDs)
assert actions_a[0].htmx_trigger != actions_b[0].htmx_trigger

# JS callbacks are different (different prefixes)
assert actions_a[2].js_callback == "text_jumpPageUp"
assert actions_b[2].js_callback == "vad_jumpPageUp"
print("Multi-instance action tests passed!")

Multi-instance action tests passed!


## Consumer Integration Example

The consumer extends the library's actions with workflow-specific actions
and assembles the full `ZoneManager`.

In [ ]:
from cjm_fasthtml_keyboard_navigation.core.modes import KeyboardMode
from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager

# Consumer creates zone and gets library nav actions
_reset_prefix_counter()
config = CardStackConfig()
ids = CardStackHtmlIds(prefix=config.prefix)
btn_ids = CardStackButtonIds(prefix=config.prefix)

zone = create_card_stack_focus_zone(ids)
nav_actions = create_card_stack_nav_actions(
    zone.id, btn_ids, config, disable_in_modes=("split",)
)

# Consumer adds their own actions
consumer_actions = (
    KeyAction(
        key="Enter",
        htmx_trigger="my-split-btn",
        not_modes=("split",),
        description="Enter split mode",
    ),
    KeyAction(
        key="Backspace",
        htmx_trigger="my-merge-btn",
        not_modes=("split",),
        description="Merge with previous",
    ),
)

# Consumer defines modes
split_mode = KeyboardMode(
    name="split",
    navigation_override=ScrollOnly(),
    indicator_text="Split Mode",
)

# Consumer assembles full manager
manager = ZoneManager(
    zones=(zone,),
    actions=nav_actions + consumer_actions,
    modes=(split_mode,),
    prev_zone_key="",
    next_zone_key="",
    state_hidden_inputs=True,
)

assert len(manager.zones) == 1
assert len(manager.actions) == 12  # 10 library + 2 consumer
assert len(manager.modes) == 1
print(f"ZoneManager assembled: {len(manager.actions)} actions, {len(manager.modes)} modes")

ZoneManager assembled: 12 actions, 1 modes


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()